## Overview

We're going to continue to improve on our model from the previous objective. So far, we have used a pipeline with a vectorizer and a classifier. Another advantage of a pipeline is that it's easy to add additional tasks. For this objective, we're going to look at a technique called *latent semantic analysis* (LSA). It's also referred to as latent semantic indexing (LSI) and the terms are often used interchangeably.

When we're doing this type of analysis, we're looking to find a set of concepts that are common to all of the documents in our corpus. A word count (or some type of vector representation) is determined for each document. Document similarity can then be determined by calculating the cosine similarity between two document vectors (cosine similarity is the normalized dot product).

### Singular Value Decomposition (SVD)

We won't go into all of the detailed math here, but we'll just try to summarize the main ideas behind using singular value composition. As we have seen in previous modules, we can create a matrix which represents the words in a corpus. In the previous objective, we have a list of yelp reviews and are considering each one a document. FOr each review (document) we calculate the tf-idf vector. The resulting matrix has rows that correspond the the words in the corpus and the columns are each individual document. For a large corpus, this matrix could also be large; we don't necessarily need all of the information contained within it. 

To find the important "parts" of the matrix, we can use SVD to reduce the number of rows (words) while still preserving enough information for later comparisons, like the cosine similarity.

In the following example, we'll so a latent semantic analysis using the Scikit-learn `TruncatedSVD` transformer. This tool works on tf-idf matrices as returned by the vectorizers. When we apply the transformer to this type of matrix, it is known as latent semantic analysis (LSA).

## Follow Along

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

# Read in the locally saved file from the link above

df_yelp = pd.read_csv('yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df_yelp.head()

# Create the features and target
sentences = df_yelp['sentence']
y = df_yelp['label']

# Instantiate the tf-idf vectorizer
vectorizer = TfidfVectorizer(stop_words='english', ngram_range = (2,2))

# Instantiate the classifier (defaults)
classifier = LogisticRegression(solver='lbfgs')

# Instantiate the LSA (SVD) algorithm (defaults)
svd = TruncatedSVD()

Now we can add the SVD part to our pipeline; we'll separate it from the classifier part and call the combination of the vectorizer and SVD the "lsa" piece.

In [2]:
# Create the pipelines
from sklearn.pipeline import Pipeline

# LSA part
lsa = Pipeline([('vect', vectorizer), ('svd', svd)])

# Combine into one pipeline
pipe = Pipeline([('lsa', lsa), ('clf', classifier)])

# Define the parameter space for the grid search
parameters = {
    'lsa__svd__n_components': (100,250),
    'lsa__vect__max_df': (0.9, 1.0), # max document frequency
}

# Implement a grid search with cross-validation
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(sentences, y);

# Display the best score from the grid-search
grid_search.best_score_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    3.4s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.4s finished


0.594

In the previous objective, we achieved an accuracy of 0.61 and we're at about the same here. Our analysis didn't seem to benefit too much from adding in LSA/SVD. But, we did use a relatively small set of labeled sentences and the resulting matrix likely didn't contain too much extra information that needed to be "decomposed".

## Challenge

In the UCI Sentiment Labeled Sentences dataset, there are two other sources that could be included in the analysis: Amazon and IMDB. You can use a dataset that contains all three of these sources or just select one of them. With that dataset, try running the analysis above. Did it make any difference to include SVD?

## Additional Resources

* [Latent Semantic Analysis](https://technowiki.wordpress.com/2011/08/27/latent-semantic-analysis-lsa-tutorial/)
* [Deep Learning Book: Singular Value Decomposition](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.8-Singular-Value-Decomposition/)
* [Scikit-learn: Truncated SVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)
* [UCI: Sentiment Labeled Sentences](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences)